<a href="https://colab.research.google.com/github/leeJoonseob/Rabble/blob/develop/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import matplotlib.font_manager as fm

폰트 설정을 위한 파일 다운로드

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
font_list = [f.name for f in fm.fontManager.ttflist]
print(font_list)

['STIXGeneral', 'STIXSizeOneSym', 'DejaVu Serif', 'cmss10', 'DejaVu Sans Display', 'cmsy10', 'DejaVu Sans Mono', 'DejaVu Serif', 'STIXNonUnicode', 'cmr10', 'STIXSizeTwoSym', 'DejaVu Sans Mono', 'STIXGeneral', 'STIXNonUnicode', 'cmex10', 'STIXSizeFourSym', 'cmb10', 'DejaVu Serif Display', 'STIXNonUnicode', 'DejaVu Sans Mono', 'DejaVu Sans', 'STIXSizeTwoSym', 'STIXSizeOneSym', 'DejaVu Serif', 'DejaVu Serif', 'DejaVu Sans', 'STIXSizeFiveSym', 'cmmi10', 'cmtt10', 'STIXSizeThreeSym', 'STIXGeneral', 'STIXSizeFourSym', 'DejaVu Sans', 'DejaVu Sans Mono', 'STIXNonUnicode', 'DejaVu Sans', 'STIXSizeThreeSym', 'STIXGeneral', 'Liberation Serif', 'Liberation Sans', 'Liberation Sans Narrow', 'Liberation Serif', 'Liberation Sans', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Serif', 'Humor Sans', 'Liberation Sans Narrow', 'Liberation Mono', 'Liberation Mono', 'Liberation Mono', 'Liberation Mono', 'Liberation Sans Narrow', 'Liberation Serif', 'Liberation Sans']


In [8]:
spam_virus = pd.read_csv("/spma_virus.csv")

In [9]:
print(spam_virus.head())

         수신일자   수신시간  메일종류                                               메일제목  \
0  2023-01-01  00:03    스팸  Add These Leaves To Every Drink And Speed Up W...   
1  2023-01-01  00:31    스팸                                     Wire and Cable   
2  2023-01-01  00:38    스팸   Poop Instantly No Matter How Constipated You Are   
3  2023-01-01  02:29  바이러스                                      С Наступающим   
4  2023-01-01  02:42    스팸                       jay, Do not just watch, shop   

                                                  첨부  
0                                                 없음  
1                                                 없음  
2                                                 없음  
3  Предложение.docx (18.7 K), Offer.docx (16.8 K)...  
4                                                 없음  


In [10]:
spam_virus['수신일자'] = pd.to_datetime(spam_virus['수신일자'])
spam_virus['수신일자'] = spam_virus['수신일자'].astype(int) / 10**9

In [11]:
spam_virus['수신시간'] = spam_virus['수신시간'].apply(lambda x: f"{x}:00" if len(x.split(':')) == 2 else x)
spam_virus['수신시간'] = pd.to_timedelta(spam_virus['수신시간']).dt.total_seconds()

In [12]:
mapping = {'스팸': 1, '바이러스': 2}

spam_virus['메일종류'] = spam_virus['메일종류'].map(mapping)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화 수행
vectorizer = TfidfVectorizer(max_features=100)  # 최대 100개의 특성을 사용
tfidf_matrix = vectorizer.fit_transform(spam_virus['메일제목'])

# TF-IDF 결과를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=spam_virus.index
)

In [14]:
# '메일제목' 열을 삭제하고 TF-IDF 데이터로 대체
spam_virus = spam_virus.drop('메일제목', axis=1)
spam_virus = pd.concat([spam_virus, tfidf_df], axis=1)

In [15]:
spam_virus['첨부'] = spam_virus['첨부'].map(lambda x: 1 if x != '없음' else 0)

In [16]:
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 12

In [17]:
sns.set(rc={'figure.figsize':(100,100)})
correlation_matrix = spam_virus.corr().round(2)
sns.heatmap(data=correlation_matrix, annot=True)
plt.show()


/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 49888 (\N{HANGUL SYLLABLE SIN}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 49884 (\N{HANGUL SYLLABLE SI}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 44036 (\N{HANGUL SYLLABLE GAN}) missing from current font.
  fig.canvas.draw()
/usr/local/lib/python3.10/dist-packages/seaborn/ut

In [22]:
tfidf_columns = spam_virus.columns[:]

# 각 단어(열)의 TF-IDF 값 합계 계산
word_importance = spam_virus[tfidf_columns].sum().sort_values(ascending=False)

# 상위 20개 단어 출력
print(word_importance.head(60))

수신일자           1.279422e+14
수신시간           3.285304e+09
메일종류           8.036000e+04
첨부             8.377000e+03
광고             3.783951e+03
your           3.032983e+03
you            2.896191e+03
to             2.459799e+03
for            1.974625e+03
re             1.885752e+03
회원             1.820463e+03
하셨습니다          1.809138e+03
가입을            1.808855e+03
koreastamp     1.807832e+03
께서             1.806479e+03
this           1.547547e+03
안내             1.509714e+03
and            1.442232e+03
2023           1.435060e+03
the            1.375420e+03
recorded       1.353671e+03
주식             1.314638e+03
or             1.288645e+03
posa           1.285098e+03
order          1.255600e+03
cooperation    1.253102e+03
предложение    1.192000e+03
new            1.172249e+03
kr             1.145531e+03
payment        1.079528e+03
in             1.077877e+03
now            1.069680e+03
of             1.058778e+03
한번             9.609208e+02
has            9.246858e+02
offer          8.993

In [20]:
print(spam_virus.shape)

(75686, 104)


In [23]:
spam_virus = spam_virus[["수신일자","수신시간","메일종류","첨부","광고","회원","가입을","주식","posa","предложение","payment","한번","드립니다","해외선물","선물","선착순","보장","해드릴께요","태국","혼자","보실래요","밤에","신나게","마세요"]]